## Building A Chatbot
Here, We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [2]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key='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'
#os.getenv("GROQ_API_KEY")



In [3]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7db688d8ac10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7db688d8dc10>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Samara and I am a AI ML Engineer")])

AIMessage(content="Hi Samara!\n\nThat's awesome! It's always great to meet another AI/ML enthusiast.  \n\nWhat kind of projects are you working on these days?\n\nI'm eager to hear more about your work. 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 22, 'total_tokens': 75, 'completion_time': 0.096363636, 'prompt_time': 0.002166905, 'queue_time': 0.23593199, 'total_time': 0.098530541}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-433b5401-47bb-49f2-bc82-810eb52e6d94-0', usage_metadata={'input_tokens': 22, 'output_tokens': 53, 'total_tokens': 75})

In [6]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi, My name is Samara and I am a AI ML Engineer"),
        AIMessage(content="Hello Samara! It's nice to meet you. \n\nAs a AI ML Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You are Samara, and you are an AI/ML Engineer! 😊  \n\nThat's what you told me when we first met.  Is there anything else you'd like me to remember about you?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 97, 'total_tokens': 143, 'completion_time': 0.083636364, 'prompt_time': 0.005292867, 'queue_time': 0.28505117300000005, 'total_time': 0.088929231}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-98714ff9-a6aa-434d-89f4-5697e440552f-0', usage_metadata={'input_tokens': 97, 'output_tokens': 46, 'total_tokens': 143})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [7]:
!pip install langchain_community

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [8]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [9]:
config={"configurable":{"session_id":"chat1"}}

In [11]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Samara and I am a AI ML Engineer")],
    config=config
)

In [12]:
response.content

"Hi Samara, it's great to meet you!  \n\nBeing an AI/ML Engineer is a really important role in shaping the future. What are you most passionate about in your work?  \n\nDo you have any interesting projects you're working on right now that you'd like to tell me about?  I'm always curious to hear about new developments in the field. 😊 \n\n"

In [13]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Samara.  \n\nI remember you told me when we first met! 😊  \n\n\n\n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 201, 'total_tokens': 224, 'completion_time': 0.041818182, 'prompt_time': 0.01231626, 'queue_time': 0.23104312300000002, 'total_time': 0.054134442}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-5914daec-71ef-4df7-a1c0-24e577e285f7-0', usage_metadata={'input_tokens': 201, 'output_tokens': 23, 'total_tokens': 224})

In [14]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me your name, I'd be happy to use it! 😊\n"

In [15]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hey John! It's nice to meet you. What can I do for you today? 😊  \n\n"

In [16]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"Your name is John, that's what you told me! 😊  \n\nIs there anything else I can help you with? \n"

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [17]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [18]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Krish")]})

AIMessage(content="Hi Krish! \n\nIt's nice to meet you.\n\nWhat can I do for you today? I'm ready to answer any questions you have to the best of my ability! 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 31, 'total_tokens': 76, 'completion_time': 0.081818182, 'prompt_time': 0.002379526, 'queue_time': 0.234850488, 'total_time': 0.084197708}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-110036fe-38be-428f-aedd-f66e4144cc7d-0', usage_metadata={'input_tokens': 31, 'output_tokens': 45, 'total_tokens': 76})

In [19]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [21]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Samara")],
    config=config
)

response

AIMessage(content="Hi Samara! It's great to meet you too. 😊  \n\nWhat can I do for you today?  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 87, 'total_tokens': 114, 'completion_time': 0.049090909, 'prompt_time': 0.00510802, 'queue_time': 0.23605352400000001, 'total_time': 0.054198929}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-b0e4d0ee-9ba1-492d-8c6f-dea2e923ab7e-0', usage_metadata={'input_tokens': 87, 'output_tokens': 27, 'total_tokens': 114})

In [22]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Samara.  😊 \n\nIs there anything else I can help you with?\n'

In [23]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [24]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Krish")],"language":"Hindi"})
response.content

'नमस्ते कृष्ण! \n\nमुझे बहुत खुशी है कि आपने मुझसे बातचीत शुरू की है। आप मुझसे जो भी पूछना चाहें, मैं अपनी पूरी कोशिश करूँगा कि आपको सही उत्तर दूं।  \n\n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [25]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [27]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Samara")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते समारा! 😊  आपका स्वागत है! \n\nक्या मैं आपकी कोई मदद कर सकता हूँ?  \n'

In [28]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [29]:
response.content

'आपका नाम समारा है।  \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [33]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [ ]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

In [40]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As an AI, I don't have access to your personal preferences like your favorite ice cream flavor.  \n\nWhat's your favorite ice cream? 😊🍦\n"

In [41]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "whats 2 + 2" 😊  \n\n\n\n'

In [42]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [43]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I don't have access to past conversations or any personal information about you, including your name.  \n\nIf you'd like to tell me your name, I'd be happy to know! 😊  \n\n"

In [44]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊  Just let me know what it is. \n\n"